# Quantum Process Tomography

* **Author:** Christopher J. Wood (cjwood@us.ibm.com)
* **Last Updated:** May 15, 2018 

In [1]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit
from qiskit import QuantumRegister, QuantumCircuit
from qiskit.tools.qi.qi import state_fidelity, outer

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo

## 1-qubit process tomography example

In [8]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, 'local_unitary_simulator')
ideal_unitary = job.result().get_data(circ).get('unitary')
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, 'local_qasm_simulator', shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('X',), ('Zp',)): {'0': 2000},
 (('Y',), ('Zp',)): {'0': 1040, '1': 960},
 (('Z',), ('Zp',)): {'0': 1005, '1': 995},
 (('X',), ('Zm',)): {'1': 2000},
 (('Y',), ('Zm',)): {'0': 1010, '1': 990},
 (('Z',), ('Zm',)): {'0': 972, '1': 1028},
 (('X',), ('Xp',)): {'0': 999, '1': 1001},
 (('Y',), ('Xp',)): {'0': 1005, '1': 995},
 (('Z',), ('Xp',)): {'0': 2000},
 (('X',), ('Yp',)): {'0': 1009, '1': 991},
 (('Y',), ('Yp',)): {'1': 2000},
 (('Z',), ('Yp',)): {'0': 1027, '1': 973}}

In [9]:
# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts)

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.mle_fit(data, basis, trace=2)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.cvx_fit(data, basis,
                        trace_preserving=True,
                        trace=2)

print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.007817983627319336
fit fidelity: 0.9969457800217439

CVXOPT Fitter
fit time: 0.07792878150939941
fit fidelity: 1.0000356207534116


## 2-Qubit entangling circuit

In [10]:
# Bell-state entangling circuit
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.h(q[0])
circ.cx(q[0], q[1])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, 'local_unitary_simulator')
ideal_unitary = job.result().get_data(circ).get('unitary')
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, 'local_qasm_simulator', shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)

# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts)


t = time.time()
choi_mle = tomo.mle_fit(data, basis,PSD=False, trace=4)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_mle / 4))

t = time.time()
choi_cvx = tomo.cvx_fit(data, basis,
                        PSD=True,
                        trace_preserving=True,
                        solver='CVXOPT')
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_cvx / 4))

MLE Fitter
fit time: 0.07346296310424805
fit fidelity: 0.9985959480281283

CVXOPT Fitter
fit time: 1.4449901580810547
fit fidelity: 0.9999260271222195


## Using SIC-POVM preparation basis

In [13]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, 'local_unitary_simulator')
ideal_unitary = job.result().get_data(circ).get('unitary')
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.tomography_circuits(circ, q, prep_labels='SIC', prep_basis='SIC')
job = qiskit.execute(qpt_circs, 'local_qasm_simulator', shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('X',), ('S0',)): {'0': 2000},
 (('Y',), ('S0',)): {'0': 1036, '1': 964},
 (('Z',), ('S0',)): {'0': 988, '1': 1012},
 (('X',), ('S1',)): {'0': 610, '1': 1390},
 (('Y',), ('S1',)): {'0': 989, '1': 1011},
 (('Z',), ('S1',)): {'0': 1942, '1': 58},
 (('X',), ('S2',)): {'0': 694, '1': 1306},
 (('Y',), ('S2',)): {'0': 1806, '1': 194},
 (('Z',), ('S2',)): {'0': 553, '1': 1447},
 (('X',), ('S3',)): {'0': 690, '1': 1310},
 (('Y',), ('S3',)): {'0': 189, '1': 1811},
 (('Z',), ('S3',)): {'0': 520, '1': 1480}}

In [15]:
# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts, prep_basis='SIC')

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.mle_fit(data, basis, trace=2)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.cvx_fit(data, basis, trace_preserving=True)

print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.013880014419555664
fit fidelity: 0.9875806749111135

CVXOPT Fitter
fit time: 0.10106420516967773
fit fidelity: 0.9943818659624815
